## Cleaning and preprocessing the data for sentiment analysis

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import contractions

In [2]:
tweets1 = pd.read_csv("bitcointweets_2020_2021.csv")
tweets2 = pd.read_csv("bitcointweets_2021_may22.csv")

In [3]:
tweets1.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/JWWeatherman_/status/13447...,2020-12-31 23:54:04+00:00,Just helped someone remove 3 #bitcoin from @co...,Just helped someone remove 3 #bitcoin from @co...,1344793979049701380,"{'_type': 'snscrape.modules.twitter.User', 'us...",53,44,618,...,NaN,NaN,NaN,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,['bitcoin'],NaN
1,snscrape.modules.twitter.Tweet,https://twitter.com/Wealth_Theory/status/13447...,2020-12-31 23:36:34+00:00,Well 2020 was a hell of a year. I’ve never see...,Well 2020 was a hell of a year. I’ve never see...,1344789575370207234,"{'_type': 'snscrape.modules.twitter.User', 'us...",32,34,579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN
2,snscrape.modules.twitter.Tweet,https://twitter.com/GoingParabolic/status/1344...,2020-12-31 23:34:34+00:00,Thanks for all the #Bitcoin @BlockFi and Happy...,Thanks for all the #Bitcoin @BlockFi and Happy...,1344789071210704902,"{'_type': 'snscrape.modules.twitter.User', 'us...",91,17,651,...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,NaN,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,['Bitcoin'],NaN
3,snscrape.modules.twitter.Tweet,https://twitter.com/AltcoinPsycho/status/13447...,2020-12-31 23:23:47+00:00,"In the next alt season, there will be a lot of...","In the next alt season, there will be a lot of...",1344786358422839297,"{'_type': 'snscrape.modules.twitter.User', 'us...",52,25,569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,snscrape.modules.twitter.Tweet,https://twitter.com/davidgokhshtein/status/134...,2020-12-31 23:19:18+00:00,I will give someone $100 in 24 hours. All you ...,I will give someone $100 in 24 hours. All you ...,1344785229354070018,"{'_type': 'snscrape.modules.twitter.User', 'us...",889,1505,1074,...,NaN,NaN,NaN,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,"['crypto', 'Bitcoin', 'PAC']","['XRP', 'VET', 'ETH', 'VIDT', 'XLM', 'XDN', 'Z..."


### Joining the two datasets into a single large dataset

In [4]:
data = pd.concat([tweets2,tweets1])
data

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/AurelienOhayon/status/1520...,2022-04-30 23:55:42+00:00,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,1520552505050075137,"{'_type': 'snscrape.modules.twitter.User', 'us...",224,190,1302,...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['BITCOIN', 'BTC', 'CRYPTO']",NaN
1,snscrape.modules.twitter.Tweet,https://twitter.com/TheMoonCarl/status/1520551...,2022-04-30 23:49:53+00:00,When do you think #Bitcoin will hit $2T in mar...,When do you think #Bitcoin will hit $2T in mar...,1520551043150626816,"{'_type': 'snscrape.modules.twitter.User', 'us...",420,74,692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN
2,snscrape.modules.twitter.Tweet,https://twitter.com/VitalikButerin/status/1520...,2022-04-30 23:49:27+00:00,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...","@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",1520550934354440192,"{'_type': 'snscrape.modules.twitter.User', 'us...",199,120,1317,...,NaN,NaN,NaN,1.520226e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN
3,snscrape.modules.twitter.Tweet,https://twitter.com/MonetaryFrame/status/15205...,2022-04-30 23:32:03+00:00,KYC is truly an attack on #Bitcoin,KYC is truly an attack on #Bitcoin,1520546552661565440,"{'_type': 'snscrape.modules.twitter.User', 'us...",51,51,435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN
4,snscrape.modules.twitter.Tweet,https://twitter.com/MartiniGuyYT/status/152054...,2022-04-30 23:23:56+00:00,"If #Bitcoin doesn't reach $50,000 by May 20th...","If #Bitcoin doesn't reach $50,000 by May 20th...",1520544509305761792,"{'_type': 'snscrape.modules.twitter.User', 'us...",1389,1608,8639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10628,snscrape.modules.twitter.Tweet,https://twitter.com/DavisMattek/status/1212211...,2020-01-01 03:18:14+00:00,I’d like my last tweet of the decade to be:\n\...,I’d like my last tweet of the decade to be:\n\...,1212211401630113793,"{'_type': 'snscrape.modules.twitter.User', 'us...",70,127,2237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10629,snscrape.modules.twitter.Tweet,https://twitter.com/ErikVoorhees/status/121220...,2020-01-01 02:35:31+00:00,Here’s to 2020 and the next ten years 🎉 May th...,Here’s to 2020 and the next ten years 🎉 May th...,1212200650739789824,"{'_type': 'snscrape.modules.twitter.User', 'us...",49,159,995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['bitcoin'],NaN
10630,snscrape.modules.twitter.Tweet,https://twitter.com/spiralbtc/status/121219690...,2020-01-01 02:20:38+00:00,This decade was bitcoin just warming up.,This decade was bitcoin just warming up.,1212196907419947008,"{'_type': 'snscrape.modules.twitter.User', 'us...",43,370,2036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10631,snscrape.modules.twitter.Tweet,https://twitter.com/OkcashCrypto/status/121218...,2020-01-01 01:43:54+00:00,Celebrate the New Year 2020 ! \n&amp; Particip...,Celebrate the New Year 2020 ! \n&amp; Particip...,1212187660976037889,"{'_type': 'snscrape.modules.twitter.User', 'us...",312,208,237,...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,NaN,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,"['Okcash', 'Crypto', 'happynewyear2020', 'Give...",['OK']


In [5]:
data.isna().sum()

_type                   0
url                     0
date                    0
content                 0
renderedContent         0
id                      0
user                    0
replyCount              0
retweetCount            0
likeCount               0
quoteCount              0
conversationId          0
lang                    0
source                  0
sourceUrl               0
sourceLabel             0
outlinks            59492
tcooutlinks         59492
media               49842
retweetedTweet      68003
quotedTweet         65217
inReplyToTweetId    66418
inReplyToUser       66418
mentionedUsers      59659
coordinates         67521
place               67520
hashtags            24213
cashtags            64505
dtype: int64

#### the columns which has important data data doesnt contain any null value so we can skip the missing value cleaning part


## splitting the datetime dataype of date column to date and time separately


In [6]:
data['dates'] = pd.to_datetime(data['date']).dt.date

data.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,dates
0,snscrape.modules.twitter.Tweet,https://twitter.com/AurelienOhayon/status/1520...,2022-04-30 23:55:42+00:00,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,1520552505050075137,"{'_type': 'snscrape.modules.twitter.User', 'us...",224,190,1302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['BITCOIN', 'BTC', 'CRYPTO']",NaN,2022-04-30
1,snscrape.modules.twitter.Tweet,https://twitter.com/TheMoonCarl/status/1520551...,2022-04-30 23:49:53+00:00,When do you think #Bitcoin will hit $2T in mar...,When do you think #Bitcoin will hit $2T in mar...,1520551043150626816,"{'_type': 'snscrape.modules.twitter.User', 'us...",420,74,692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN,2022-04-30
2,snscrape.modules.twitter.Tweet,https://twitter.com/VitalikButerin/status/1520...,2022-04-30 23:49:27+00:00,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...","@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",1520550934354440192,"{'_type': 'snscrape.modules.twitter.User', 'us...",199,120,1317,...,NaN,NaN,1.520226e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,2022-04-30
3,snscrape.modules.twitter.Tweet,https://twitter.com/MonetaryFrame/status/15205...,2022-04-30 23:32:03+00:00,KYC is truly an attack on #Bitcoin,KYC is truly an attack on #Bitcoin,1520546552661565440,"{'_type': 'snscrape.modules.twitter.User', 'us...",51,51,435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN,2022-04-30
4,snscrape.modules.twitter.Tweet,https://twitter.com/MartiniGuyYT/status/152054...,2022-04-30 23:23:56+00:00,"If #Bitcoin doesn't reach $50,000 by May 20th...","If #Bitcoin doesn't reach $50,000 by May 20th...",1520544509305761792,"{'_type': 'snscrape.modules.twitter.User', 'us...",1389,1608,8639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Bitcoin'],NaN,2022-04-30


## removing all the redundant columns (keeping just the date and tweet)

In [7]:
data.columns

Index(['_type', 'url', 'date', 'content', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags', 'dates'],
      dtype='object')

In [8]:
# columns which aren't required 
delete_cols = ['_type', 'url','date','renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags']

data = data.drop(columns=delete_cols)
data.head()

,content,dates
0,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,2022-04-30
1,When do you think #Bitcoin will hit $2T in mar...,2022-04-30
2,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",2022-04-30
3,KYC is truly an attack on #Bitcoin,2022-04-30
4,"If #Bitcoin doesn't reach $50,000 by May 20th...",2022-04-30


# Cleaning of data 

## Lower Casing the Data
###### The idea is to convert the input text into the same casing format so that it converts ‘DATA’, ’Data’, ‘DaTa’, ‘DATa’ into ‘data’.

In [9]:
data['clean_t']=data['content'].str.lower()

## Removing contractions 
##### There are so many contractions in the text we type so to expand them we will use the contractions library.

##### The Twitter and Instagram data has so many contractions in them and if we remove the punctuations from that text then it would look like this.

##### For example, the text “I’ll eat pizza” and if we remove the punctuations them the text will look like this “I ll will eat pizza”. Here, “I ll” doesn’t hold any information to the text that’s why we use the contraction.

In [10]:
sent2 =[]

for index, row in data.iterrows():
    sent = row['clean_t']
    sent1 = contractions.fix(sent)
    sent2.append(sent1)
    
data['clean_t'] = sent2

In [11]:
data.head(20)

,content,dates,clean_t
0,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,2022-04-30,#bitcoin do not panic weak hands. part ii.\n\n...
1,When do you think #Bitcoin will hit $2T in mar...,2022-04-30,when do you think #bitcoin will hit $2t in mar...
2,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",2022-04-30,"@zengjiajun_eth bitcoin, not crypto.\n\nlike:\..."
3,KYC is truly an attack on #Bitcoin,2022-04-30,kyc is truly an attack on #bitcoin
4,"If #Bitcoin doesn't reach $50,000 by May 20th...",2022-04-30,"if #bitcoin does not reach $50,000 by may 20t..."
5,"JUST IN: #Bitcoin has fallen to $38,000.",2022-04-30,"just in: #bitcoin has fallen to $38,000."
6,Will Bitcoin be above or below $37K by the tim...,2022-04-30,will bitcoin be above or below $37k by the tim...
7,#Bitcoin should be in everyone’s retirement sa...,2022-04-30,#bitcoin should be in everyone is retirement s...
8,Warren Buffet: Bitcoin is evil.\n\nAlso Warren...,2022-04-30,warren buffet: bitcoin is evil.\n\nalso warren...
9,Are they holding back a USA ETF so insiders ca...,2022-04-30,are they holding back a usa etf so insiders ca...


line 4 , 11, 17 , etc show us clearly how contractions have been replaced

## Removing the @tagged_words and urls

In [23]:
c = []

for index,row in data.iterrows():
    text = row['clean_t']
    text = re.sub(r"(@[A-Za-z0-9._]+?)|^rt|http.+?", "", text)
    c.append(text)
    
data['clean_t'] = c

In [24]:
data.head(20)

,content,dates,clean_t
0,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,2022-04-30,#bitcoin do not panic weak hands. part ii.\n\n...
1,When do you think #Bitcoin will hit $2T in mar...,2022-04-30,when do you think #bitcoin will hit $2t in mar...
2,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",2022-04-30,"_eth bitcoin, not crypto.\n\nlike:\n\na: ""oh i..."
3,KYC is truly an attack on #Bitcoin,2022-04-30,kyc is truly an attack on #bitcoin
4,"If #Bitcoin doesn't reach $50,000 by May 20th...",2022-04-30,"if #bitcoin does not reach $50,000 by may 20t..."
5,"JUST IN: #Bitcoin has fallen to $38,000.",2022-04-30,"just in: #bitcoin has fallen to $38,000."
6,Will Bitcoin be above or below $37K by the tim...,2022-04-30,will bitcoin be above or below $37k by the tim...
7,#Bitcoin should be in everyone’s retirement sa...,2022-04-30,#bitcoin should be in everyone is retirement s...
8,Warren Buffet: Bitcoin is evil.\n\nAlso Warren...,2022-04-30,warren buffet: bitcoin is evil.\n\nalso warren...
9,Are they holding back a USA ETF so insiders ca...,2022-04-30,are they holding back a usa etf so insiders ca...


## Removing the Punctuation and Numbers

##### The punctuation removal process will help to treat each text equally. For example, the word data and data! are treated equally after the process of removal of punctuations.

In [25]:
x = [ ]
for index, row in data.iterrows():
    sentence = row['clean_t']
    sentence = re.sub('[^a-z]',' ',sentence)
    x.append(sentence)

data['clean_t'] = x

In [26]:
data.head(20)

,content,dates,clean_t
0,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,2022-04-30,bitcoin do not panic weak hands part ii b...
1,When do you think #Bitcoin will hit $2T in mar...,2022-04-30,when do you think bitcoin will hit t in mar...
2,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",2022-04-30,eth bitcoin not crypto like a oh i am ...
3,KYC is truly an attack on #Bitcoin,2022-04-30,kyc is truly an attack on bitcoin
4,"If #Bitcoin doesn't reach $50,000 by May 20th...",2022-04-30,if bitcoin does not reach by may t...
5,"JUST IN: #Bitcoin has fallen to $38,000.",2022-04-30,just in bitcoin has fallen to
6,Will Bitcoin be above or below $37K by the tim...,2022-04-30,will bitcoin be above or below k by the tim...
7,#Bitcoin should be in everyone’s retirement sa...,2022-04-30,bitcoin should be in everyone is retirement s...
8,Warren Buffet: Bitcoin is evil.\n\nAlso Warren...,2022-04-30,warren buffet bitcoin is evil also warren b...
9,Are they holding back a USA ETF so insiders ca...,2022-04-30,are they holding back a usa etf so insiders ca...


## Removing Emojis


In [27]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"U0001F600-U0001F64F"  # emoticons
                           u"U0001F300-U0001F5FF"  # symbols & pictographs
                           u"U0001F680-U0001F6FF"  # transport & map symbols
                           u"U0001F1E0-U0001F1FF"  # flags (iOS)
                           u"U00002702-U000027B0"
                           u"U000024C2-U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [28]:
y = [ ]
for index, row in data.iterrows():
    sentence = row['clean_t']
    sentence = remove_emoji(sentence)
    y.append(sentence)

data['clean_t'] = y

In [29]:
data.head(100)

,content,dates,clean_t
0,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,2022-04-30,bitcoin do not panic weak hands part ii b...
1,When do you think #Bitcoin will hit $2T in mar...,2022-04-30,when do you think bitcoin will hit t in mar...
2,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",2022-04-30,eth bitcoin not crypto like a oh i am ...
3,KYC is truly an attack on #Bitcoin,2022-04-30,kyc is truly an attack on bitcoin
4,"If #Bitcoin doesn't reach $50,000 by May 20th...",2022-04-30,if bitcoin does not reach by may t...
...,...,...,...
95,Lots of #Altcoins breaking major support level...,2022-04-30,lots of altcoins breaking major support level...
96,#BITCOIN MONTHLY BULLISH DIVERGENCE! 🔥👇 https:...,2022-04-30,bitcoin monthly bullish divergence t c...
97,Volatility is the reason to buy more #Bitcoin\...,2022-04-30,volatility is the reason to buy more bitcoin ...
98,With Bitcoin now being made legal tender in bo...,2022-04-30,with bitcoin now being made legal tender in bo...


## Removing stopwords

In [30]:
stop = stopwords.words('english')
s = []
for index, row in data.iterrows():
    text = row['clean_t']
    text = " ".join([word for word in text.split() if word not in (stop)])
    s.append(text)
    
data['clean_t']=s

In [31]:
data.head(20)

,content,dates,clean_t
0,#BITCOIN DON'T PANIC WEAK HANDS. PART II.\n\n#...,2022-04-30,bitcoin panic weak hands part ii btc crypto co...
1,When do you think #Bitcoin will hit $2T in mar...,2022-04-30,think bitcoin hit market capitalization
2,"@zengjiajun_eth Bitcoin, not crypto.\n\nLike:\...",2022-04-30,eth bitcoin crypto like oh really bitcoins lov...
3,KYC is truly an attack on #Bitcoin,2022-04-30,kyc truly attack bitcoin
4,"If #Bitcoin doesn't reach $50,000 by May 20th...",2022-04-30,bitcoin reach may th give one lucky follower l...
5,"JUST IN: #Bitcoin has fallen to $38,000.",2022-04-30,bitcoin fallen
6,Will Bitcoin be above or below $37K by the tim...,2022-04-30,bitcoin k time monday co tghdelm
7,#Bitcoin should be in everyone’s retirement sa...,2022-04-30,bitcoin everyone retirement savings plan alrea...
8,Warren Buffet: Bitcoin is evil.\n\nAlso Warren...,2022-04-30,warren buffet bitcoin evil also warren buffett...
9,Are they holding back a USA ETF so insiders ca...,2022-04-30,holding back usa etf insiders buy bitcoin cheap


In [36]:
data = data[['dates','clean_t']]
data.head()

,dates,clean_t
0,2022-04-30,bitcoin panic weak hands part ii btc crypto co...
1,2022-04-30,think bitcoin hit market capitalization
2,2022-04-30,eth bitcoin crypto like oh really bitcoins lov...
3,2022-04-30,kyc truly attack bitcoin
4,2022-04-30,bitcoin reach may th give one lucky follower l...


In [37]:
data.to_csv("clean_tweets.csv")